In [1]:
from qiskit import QuantumCircuit, Aer, execute, IBMQ
from qiskit.providers import JobStatus
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel
from qiskit.tools.visualization import plot_histogram
from math import atan2, sqrt, pi
import import_ipynb
import probabilities as prob

importing Jupyter notebook from probabilities.ipynb


In [5]:
statesStayHome, statesTests, statesCases = prob.get_lesser_model_states()
basic_graph = prob.get_conditional_probability(statesStayHome, statesTests, statesCases)

"""
Order of Cases conditional probabilities in graph: 
P(C=0|A=0,B=0), P(C=1|A=0,B=0), P(C=0|A=0,B=1), P(C=1|A=0,B=1), P(C=0|A=1,B=0), P(C=1|A=1,B=0), P(C=0|A=1,B=1), P(C=1|A=1,B=1)
From Fig 4 in https://arxiv.org/pdf/2004.14803.pdf -- 
our order is first col first row, first col second row, second col first row, etc. 
so the first two probabilities add up to one, the third and fourth add up to one, etc. 
"""
print(basic_graph)
oil_graph = {'IR': ([], [.75, .25]), # From 2004.14803, Fig 10
         'SM': (['IR'], [.3, .7, .8, .2]), #P(0|!A), P(1|!A), P(0|A), P(1|A)
         'OI': ([], [.6, .4]),
         'SP': (['OI', 'SM'], [.9, .1, .5, .5, .4, .6, .2, .8])
        }


TypeError: 'numpy.int64' object is not iterable

In [ ]:
#TODO assertAlmostEqual(angle_from_probability(.8,.2), 2.2143)
def angle_from_probability(p1, p0):
    # From 2004.14803 equation 20
    return 2 * atan2(sqrt(p1), sqrt(p0))


In [6]:
import math

def find_num_states(numprobs, in_edges):
    if in_edges == 0: # if parent node
        numstates = numprobs
    elif in_edges > 0: # if child node
        numstates = (numprobs/(2**(in_edges+1))) * 2
    return numstates

def find_numstates_parents(graph, parentvarnames):
    """Eqn 21 from https://arxiv.org/pdf/2004.14803.pdf"""
    numstates_parents = 0
    for i in range(len(parentvarnames)):
        parent = parentvarnames[i]
        parentprobs = graph[parent]
        numprobs = len(parentprobs[1]) # number of states if parent node, number of conditional states if child node
        in_edges = len(graph[parent][0]) # aka num of parents of this parent
        numstates_parents += find_num_states(numprobs, in_edges)
    return numstates_parents
    
    
def num_qbits_needed_general(graph): #THIS FUNCTION DOES NOT WORK FOR 3 NODE CASE BUT I THINK IT DOES MATCH EQN 22.....
    sumterm = 0
    ancillaterms = []
    varnum = 0
    for var in graph:
        probs = graph[var]
        numprobs = len(probs[1]) # number of states if parent node, number of conditional states if child node
        in_edges = len(graph[var][0]) # aka num of parents
        numstates = find_num_states(numprobs, in_edges)
        sumterm += math.ceil(np.log2(numstates))
        if in_edges > 0:
            parentvarnames = graph[var][0] # list of parent names
            numstates_parents = find_numstates_parents(graph, parentvarnames)
            ancillaterm = numstates_parents/2 + math.ceil(np.log2(numstates)) - 1 #MAYBE THERE SHOULD NOT BE THE LOG2(N) HERE!?!? 
            ancillaterms.append(ancillaterm)
    qbits = sumterm + max(ancillaterms) # equation (22)
    cbits = len(graph) # number of measurements
    return (qbits, cbits)

def num_qbits_needed(graph):
    max_edges = -1 
    for state in graph:
        in_edges = len(graph[state][0]) # aka num of parents
        if in_edges > max_edges:
            max_edges = in_edges
    
    qbits = len(graph) + max_edges - 1 # equation (15)
    cbits = len(graph) # number of measurements
    return (qbits, cbits)

In [ ]:
def run_circuit(circuit, output_file='results', draw_circuit=True, use_sim=True, use_noise=False, use_qcomp=False, shots=1024):
    if draw_circuit:
        #print(circuit.draw())
        %config InlineBackend.figure_format
        circuit.draw(output='mpl')

    if use_noise or use_qcomp:
        IBMQ.load_account()
        provider = IBMQ.get_provider('ibm-q')
        qcomp = provider.get_backend('ibmq_16_melbourne')

    if use_sim:
        simulator = Aer.get_backend('qasm_simulator')
        if use_noise:
            noise_model = NoiseModel.from_backend(qcomp)
            basis_gates = noise_model.basis_gates
            coupling_map = qcomp.configuration().coupling_map
            job = execute(circuit, backend=simulator, coupling_map=coupling_map, noise_model=noise_model, basis_gates=basis_gates, shots=shots)
        else:
            job = execute(circuit, backend=simulator, shots=shots)
        print(job.result().get_counts())
        plot_histogram(job.result().get_counts()).savefig(output_file+"-sim.png")


    if use_qcomp:
        job = execute(circuit, backend=qcomp, shots=shots)
        job_monitor(job)
        if(job.status() == JobStatus.ERROR):
            print("Error: Check with IBMQ")
        print(job.result().get_counts())
        plot_histogram(job.result().get_counts()).savefig(output_file+'-qcomp.png')
    return job.result()

In [ ]:
# XXX Can we generalize this to C^nY gates
# ex: add_ccy(qc, [0,1,2], math.pi/3, 3)
def add_ccy(circuit, qbits, angle, ancillabit):
    # XXX Should we use this link instead of this function:
    # https://qiskit.org/documentation/tutorials/circuits_advanced/1_advanced_circuits.html
    circuit.barrier() # for visualization
    circuit.ccx(qbits[0], qbits[1], ancillabit)
    
    circuit.ry(angle/2,qbits[2])
    circuit.cx(ancillabit, qbits[2])
    circuit.ry(-angle/2, qbits[2] )
    circuit.cx(ancillabit, qbits[2])
    
    circuit.ccx(qbits[0], qbits[1], ancillabit)
    circuit.barrier() # for visualization
    
    

In [18]:
# generalized Cny gates
def add_cny(circuit, angle, n, control_bits, target, ancilla_bits):
    '''
    circuit: existing circuit to modify
    angle: angle to rotate by (rad)
    n: number of control bits required (number of "C"s wanted for this gate)
    control_bits: list (len n)
    target: qbit you want to rotate
    ancilla_bits: list (len n-1); these are left clean (in same state as initially)
    
    Note: could also try rewriting using Qiskit's built-in mcx gates
    https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.mcx.html
    '''
    # assert some things are true
    assert n >= 2
    assert len(control_bits) == n
    assert len(ancilla_bits) == n-1
    
    # hardcode first ccx gate
    circuit.barrier() # for visualization
    circuit.ccx(control_bits[0], control_bits[1], ancilla_bits[0])
    
    # loop to create ccx gates
    for i in range(2, n):
        circuit.ccx(control_bits[i], ancilla_bits[i-2], ancilla_bits[i-1])
    
    circuit.barrier() # for visualization
    
    # add rotate block
    circuit.ry(angle/2, target)
    circuit.cx(ancilla_bits[n-2], target)
    circuit.ry(-angle/2, target)
    circuit.cx(ancilla_bits[n-2], target)
    
    circuit.barrier() # for visualization
    
    # loop to create remaining ccx gates
    for i in range(n-1, 1, -1):
        circuit.ccx(control_bits[i], ancilla_bits[i-2], ancilla_bits[i-1])
        
    # hardcode last ccx gate
    circuit.ccx(control_bits[0], control_bits[1], ancilla_bits[0])
    
    circuit.barrier() # for visualization
    
    return circuit


In [11]:
def create_circuit(graph):
    (qbits, cbits) = num_qbits_needed(graph)
    qc = QuantumCircuit(qbits, cbits)
    bit_assignment = {}
    ancillabits = []
    next_free_qbit = 0
    #Loop and find the parentless nodes, assign their rotations first
    for node in graph:
        if len(graph[node][0])==0:
            probs = graph[node][1]
            qc.ry(angle_from_probability(probs[1],probs[0]), next_free_qbit)
            bit_assignment[node] = next_free_qbit # keep track of what node is what qbit
            next_free_qbit += 1
    qc.barrier()

    #Loop and find each node that has all its parents complete and add it
    #TODO a topological sort first would save this unnessacary looping
    #XXX bad code, maybe the worst
    while True: # Loop until no states are added
        found_new = False
        for node in graph:
            if node not in bit_assignment:
                for parent in node[0]:  # get parents
                    if parent not in bit_assignment:
                        continue  # one of the parents needs to be processed first
                # Found a node we are ready to process
                found_new = True
                # TODO handle multi-bit states
                bit_assignment[node] = next_free_qbit # give it a qbit
                next_free_qbit += 1
                # FIXME hardcoded 1/2 parent, will generalize later
                if len(graph[node]) == 1:
                    print("1 parent not implemented") 
                    raise NotImplementedError
                if len(graph[node]) == 2:
                    # TODO this is gross
                    print(bit_assignment)
                    if(len(ancillabits) < 1): # need one more ancilla bit
                        ancillabits += [next_free_qbit]
                        next_free_qbit+=1
                    list_of_parents = graph[node][0]
                    list_of_probs = graph[node][1]
                    parent1_qbit = bit_assignment[list_of_parents[0]]
                    parent2_qbit = bit_assignment[list_of_parents[1]]
                    target_qbit = bit_assignment[node]
                    # |00>
                    qc.x(parent1_qbit)
                    qc.x(parent2_qbit)
                    # XXX ahhh this is horrible but will work for a demo
                    angle = angle_from_probability(list_of_probs[1], list_of_probs[0])
                    add_ccy(qc, [parent1_qbit,parent2_qbit,target_qbit], angle, ancillabits[0])
                    qc.x(parent1_qbit)
                    qc.x(parent2_qbit)
                    # |01>
                    qc.x(parent1_qbit)
                    angle = angle_from_probability(list_of_probs[3], list_of_probs[2])
                    add_ccy(qc, [parent1_qbit,parent2_qbit,target_qbit], angle, ancillabits[0])
                    qc.x(parent1_qbit)
                    # |10>
                    qc.x(parent2_qbit)
                    angle = angle_from_probability(list_of_probs[5], list_of_probs[4])
                    add_ccy(qc, [parent1_qbit,parent2_qbit,target_qbit], angle, ancillabits[0])
                    qc.x(parent2_qbit)
                    # |11>
                    angle = angle_from_probability(list_of_probs[7], list_of_probs[6])
                    add_ccy(qc, [parent1_qbit,parent2_qbit,target_qbit], angle, ancillabits[0])
        if not found_new:
            break # Completed a full loop over the nodes and all were added, done

    c_counter = 0
    for bit in bit_assignment:
        qc.measure(bit_assignment[bit], c_counter)
        c_counter += 1
    return qc

In [27]:
def generate_test_circuit():
    
    
    
    qc = QuantumCircuit(20, 5)
    
    qc = add_cny(qc, pi, 10, [0,1,2,3,4,5,6,7,8,9], 19, [10,11,12,13,14,15,16,17,18])
    
    
    return qc

In [4]:
circuit = create_circuit(basic_graph)


NameError: name 'create_circuit' is not defined

In [ ]:
result = run_circuit(circuit, "lesser_model", shots=1024*8, use_sim=True, use_qcomp=False, use_noise=False)


In [ ]:
%config InlineBackend.figure_format
circuit.draw(output='mpl')

In [ ]:
plot_histogram(result.get_counts())

In [28]:
generate_test_circuit().draw()

░                                               ░                  »
 q_0: ─░───■───────────────────────────────────────────░──────────────────»
       ░   │                                           ░                  »
 q_1: ─░───■───────────────────────────────────────────░──────────────────»
       ░   │                                           ░                  »
 q_2: ─░───┼────■──────────────────────────────────────░──────────────────»
       ░   │    │                                      ░                  »
 q_3: ─░───┼────┼────■─────────────────────────────────░──────────────────»
       ░   │    │    │                                 ░                  »
 q_4: ─░───┼────┼────┼────■────────────────────────────░──────────────────»
       ░   │    │    │    │                            ░                  »
 q_5: ─░───┼────┼────┼────┼────■───────────────────────░──────────────────»
       ░   │    │    │    │    │                       ░                  »
 q_6: ─░───┼────┼────┼────┼────┼────■──────────────────░──────────────────»
       ░   │    │    │    │    │    │                  ░                  »
 q_7: ─░───┼────┼────┼────┼────┼────┼────■─────────────░──────────────────»
       ░   │    │    │    │    │    │    │             ░                  »
 q_8: ─░───┼────┼────┼────┼────┼────┼────┼────■────────░──────────────────»
       ░   │    │    │    │    │    │    │    │        ░                  »
 q_9: ─░───┼────┼────┼────┼────┼────┼────┼────┼────■───░──────────────────»
       ░ ┌─┴─┐  │    │    │    │    │    │    │    │   ░                  »
q_10: ─░─┤ X ├──■────┼────┼────┼────┼────┼────┼────┼───░──────────────────»
       ░ └───┘┌─┴─┐  │    │    │    │    │    │    │   ░                  »
q_11: ─░──────┤ X ├──■────┼────┼────┼────┼────┼────┼───░──────────────────»
       ░      └───┘┌─┴─┐  │    │    │    │    │    │   ░                  »
q_12: ─░───────────┤ X ├──■────┼────┼────┼────┼────┼───░──────────────────»
       ░           └───┘┌─┴─┐  │    │    │    │    │   ░                  »
q_13: ─░────────────────┤ X ├──■────┼────┼────┼────┼───░──────────────────»
       ░                └───┘┌─┴─┐  │    │    │    │   ░                  »
q_14: ─░─────────────────────┤ X ├──■────┼────┼────┼───░──────────────────»
       ░                     └───┘┌─┴─┐  │    │    │   ░                  »
q_15: ─░──────────────────────────┤ X ├──■────┼────┼───░──────────────────»
       ░                          └───┘┌─┴─┐  │    │   ░                  »
q_16: ─░───────────────────────────────┤ X ├──■────┼───░──────────────────»
       ░                               └───┘┌─┴─┐  │   ░                  »
q_17: ─░────────────────────────────────────┤ X ├──■───░──────────────────»
       ░                                    └───┘┌─┴─┐ ░                  »
q_18: ─░─────────────────────────────────────────┤ X ├─░───────────────■──»
       ░                                         └───┘ ░ ┌──────────┐┌─┴─┐»
q_19: ─░───────────────────────────────────────────────░─┤ RY(pi/2) ├┤ X ├»
       ░                                               ░ └──────────┘└───┘»
 c_0: ════════════════════════════════════════════════════════════════════»
                                                                          »
 c_1: ════════════════════════════════════════════════════════════════════»
                                                                          »
 c_2: ════════════════════════════════════════════════════════════════════»
                                                                          »
 c_3: ════════════════════════════════════════════════════════════════════»
                                                                          »
 c_4: ════════════════════════════════════════════════════════════════════»
                                                                          »
«                         ░                                               ░ 
« q_0: ───────────────────░───────────────────────────